# **Child Mind Institute - Relating Physical Activity to Problematic Internet Use**

‣ GitHub page for this project 👉  [here]()

‣ An article from the institute on Summer Screen Time Use 👉  [read here](https://childmind.org/article/screen-time-and-summer/)



### The Problem at Hand 🧑‍💻 *(taken from the comptetition homepage 👉  [read here](https://www.kaggle.com/competitions/child-mind-institute-problematic-internet-use/overview))*

"In today’s digital age, problematic internet use among children and adolescents is a growing concern. Better understanding this issue is crucial for addressing mental health problems such as depression and anxiety.

Current methods for measuring problematic internet use in children and adolescents are often complex and require professional assessments. This creates access, cultural, and linguistic barriers for many families. Due to these limitations, problematic internet use is often not measured directly, but is instead associated with issues such as depression and anxiety in youth.

Conversely, physical & fitness measures are extremely accessible and widely available with minimal intervention or clinical expertise. Changes in physical habits, such as poorer posture, irregular diet, and reduced physical activity, are common in excessive technology users. We propose using these easily obtainable physical fitness indicators as proxies for identifying problematic internet use, especially in contexts lacking clinical expertise or suitable assessment tools."

**What does this mean?** The Child Mind Institute has tasked the public with building predictive machine learning models that will determine a participant's Severity Impairment Index (SII), which is a metric measuring the level of problematic internet use among children and adolescents, based on physical activity, health, and lifestyle factors. The aim is to identify signs of problematic internet use early so that preventative measures can be taken by the parent/ caretaker.

### The Data at Hand 📊

We will be working with the Child Mind Institute's *Healthy Brain Network (HBN)* dataset, a clinical sample of roughly 5,000 youth and adolescents (aged 5-22) that have undergone various clinical and research screenings for the institute. The institute has conveniently separated for us the relevant data into two distinct categories.

The first is tabular data comprising measurements from various instruments, assessments, and questionairres - in particular, it includes an assessment called the Parent-Child Internet Addiction Test (PCIAT), which is used to calculate the SII of each participant - we'll refer to this data as **feature data**. The second is time-series data collected with a wrist accelerometer given to roughly 1,000 participants to wear for up to 30 days continually while at home and going about their daily lives. The data collected from this device includes physical activity and other metrics - we'll refer to this data as **actigraphy data**.

For each we have been provided with a **train set**, on which we will train our models, and a **test set**, on which we will evaluate their performance. The train set is a full dataset that includes the SII, which is our **target variable**, and the PCIAT results used to calculate it - the test set is a much smaller collection of data that is missing this information. Our objective then is to train the models to accurately predict SII values *for each entry in the test set*.

Because the natures of the feature data and the actigraphy data are vastly different, we will use an **ensemble approach**, analyzing, feature engineering and training models separately, then merging results for the final submission. The actigraphy data is dense time-series data, which means we will get a lot of value from training a neural network on it. For the feature data, simpler baseline ML models should be appropriate. 

### Competition Evaluation 📝

The result will be evaluated based on the **quadratic weighted kappa**, which measures the agreement between two outcomes. This metric typically varies from 0 (random agreement) to 1 (complete agreement). The submission file will consist of two rows, one for id and one for SII, with an entry for each participant in the test set. An example submission has been given to us on the Kaggle page.

In [ ]:
# load pandas
import pandas as pd

# load sample submission
sample = pd.read_csv("/Users/tomragus/Library/CloudStorage/OneDrive-UCSanDiego/CMI-PIU-Model/data/sample_submission.csv")

# display sample submission
print("Sample submission")
print(f"Submission shape: {sample.shape}")
sample

### Credit 📚

Parts of this notebook, particularly in the EDA stage, were adapted from [Antonina Dolgorukova](https://datadelic.dev/)'s brilliant EDA notebooks for this competition. I highly encourage checking out her work - they are extremely in-depth and very well written.

‣ *Feature EDA 👉  [read here](https://www.kaggle.com/code/antoninadolgorukova/cmi-piu-features-eda/notebook)*

‣ *Actigraphy EDA 👉  [read here](https://www.kaggle.com/code/antoninadolgorukova/cmi-piu-actigraphy-data-eda)*


## ***Feature data***

Let's start by taking a peek into our feature data:

In [ ]:
# load train set
train = pd.read_csv("/Users/tomragus/Library/CloudStorage/OneDrive-UCSanDiego/CMI-PIU-Model/data/train.csv")

# display first 5 rows of train set
print("""Train set: where the 'features' live""")
print(f"Train shape: {train.shape}")
display(train.head())

In [ ]:
# load test set
test = pd.read_csv("/Users/tomragus/Library/CloudStorage/OneDrive-UCSanDiego/CMI-PIU-Model/data/test.csv")

# display first 5 rows of test set
print("""Test set: what we will evaluate our models on""")
print(f"Test shape: {test.shape}")
display(test.head())

It can be tricky to figure out what all of these abbreviations mean - thankfully, the Child Mind Institute was kind enough to include a **data dictionary** for this competition, which gives some extra information for each variable. Here is a little preview - [you can view the full file on the Kaggle page](https://www.kaggle.com/competitions/child-mind-institute-problematic-internet-use/data?select=data_dictionary.csv).

In [ ]:
# load data dictionary
data_dict = pd.read_csv("/Users/tomragus/Library/CloudStorage/OneDrive-UCSanDiego/CMI-PIU-Model/data/data_dictionary.csv")

# display first 5 rows of data dictionary
print("""Data Dictionary: what each feature means""")
print(f"Data Dictionary shape: {data_dict.shape}")
display(data_dict.head())

While this only gives us a snippet of the data at hand, we can see the SII and the PCIAT scores on the right side of the train set. The SII scores range from 0 to 3, with 0 representing no impairment, and 3 representing severe impairment. So, we can think of the problem as training our models to **classify** each id in the test set into one of the 4 SII classes (0, 1, 2 or 3). Classification calls for **supervised learning**.

Looking at the shape of the train set, we can see that the train set has almost 4,000 entries 🤯 this is good - the more data we have, the more finely we can tune our models.

### Loading the rest of our libraries...

In [ ]:
# load all libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import xgboost as xgb
import lightgbm as lgb
import warnings
from pandas.api.types import is_numeric_dtype, is_object_dtype, is_categorical_dtype, CategoricalDtype
from scipy import stats
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from datetime import datetime
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from matplotlib.colors import ListedColormap

warnings.filterwarnings("ignore", category=RuntimeWarning)

### **Feature Data EDA**

Let us first analyze observe the features that are related to the SII and are not present in the test set.

In [ ]:
# isolating train-only features
train_cols = set(train.columns)
test_cols = set(test.columns)
columns_not_in_test = sorted(list(train_cols - test_cols))

# addind additional information using data dictionary
data_dict[data_dict['Field'].isin(columns_not_in_test)]

Here we see each item in the Parent-Child Internet Addiction Test (PCIAT). Each question (third column) assesses a different aspect of a child's behavior related to internet use, and responses are given on a scale from 0 to 5 with the total score providing an indication of the severity of internet addiction.

We also have the season of participation in PCIAT-Season and total score in PCIAT-PCIAT_Total; so there are a total of 22 PCIAT test-related columns.

Now we will verify that the PCIAT-PCIAT_Total aligns with the corresponding SII categories by calculating the minimum and maximum scores for each SII category.

In [ ]:
# calculate max and min
pciat_min_max = train.groupby('sii')['PCIAT-PCIAT_Total'].agg(['min', 'max'])
pciat_min_max = pciat_min_max.rename(columns={'min': 'Minimum PCIAT total Score', 'max': 'Maximum total PCIAT Score'})
pciat_min_max

In [ ]:
# display range for each level of severity
data_dict[data_dict['Field'] == 'PCIAT-PCIAT_Total']['Value Labels'].iloc[0]

One thing to take into account is that some of the items in the PCIAT were ignored by participants, leading to missing values. 

In [ ]:
# display PCIAT results with missing values highlighted
train_with_sii = train[train['sii'].notna()][columns_not_in_test]
train_with_sii[train_with_sii.isna().any(axis=1)].head().style.map(lambda x: 'background-color: #FFC0CB' if pd.isna(x) else '')

**Problem:** the SII score is calculated by adding together all of the non-NA values in the test, which means a participant can ignore every question (like the second participant) and get a score of 0 - entries like this are not valid, and will skew the results. We can assume then that the SII score can sometimes be incorrect.

This can be addressed by recalculating the SII scores ourselves based on PCIAT_Total and the maximum possible score *IF* missing values were answered, ensuring that the recalculated SII meets the inteded thresholds even with missing values.

In [ ]:
# define PCIAT columns as variable
PCIAT_cols = [f'PCIAT-PCIAT_{i+1:02d}' for i in range(20)]

# recalculate SII function
def recalculate_sii(row):
    if pd.isna(row['PCIAT-PCIAT_Total']):
        return np.nan
    max_possible = row['PCIAT-PCIAT_Total'] + row[PCIAT_cols].isna().sum() * 5
    if row['PCIAT-PCIAT_Total'] <= 30 and max_possible <= 30:
        return 0
    elif 31 <= row['PCIAT-PCIAT_Total'] <= 49 and max_possible <= 49:
        return 1
    elif 50 <= row['PCIAT-PCIAT_Total'] <= 79 and max_possible <= 79:
        return 2
    elif row['PCIAT-PCIAT_Total'] >= 80 and max_possible >= 80:
        return 3
    return np.nan

train['recalc_sii'] = train.apply(recalculate_sii, axis=1)

# overwriting SII with recalc_SII
train['sii'] = train['recalc_sii']

# categorizing SII scores by severity and adding "missing" for missing values
train['complete_resp_total'] = train['PCIAT-PCIAT_Total'].where(train[PCIAT_cols].notna().all(axis=1), np.nan)
sii_map = {0: '0 (None)', 1: '1 (Mild)', 2: '2 (Moderate)', 3: '3 (Severe)'}
train['sii'] = train['sii'].map(sii_map).fillna('Missing')
sii_order = ['Missing', '0 (None)', '1 (Mild)', '2 (Moderate)', '3 (Severe)']
train['sii'] = pd.Categorical(train['sii'], categories=sii_order, ordered=True)

# dropping recalc_SII column
train.drop(columns='recalc_sii', inplace=True)

### Removing Missing Values and Duplicate Rows
Now with our SII values updated, we can remove all rows that are labeled "missing" in the SII column. *Why?* They're not useful to us, since we are training our models precisely to detect this variable.

In [ ]:
# remove rows with missing SII
initial_rows = len(train)
train = train[train['sii'] != 'Missing']
train['sii'] = train['sii'].cat.remove_unused_categories()
removed_rows = initial_rows - len(train)
print(f"Removed {removed_rows} rows with 'Missing' SII values.")
print(f"Train shape: {train.shape}")

# remove duplicate rows
duplicate_count = train.duplicated().sum()
print(f"Duplicate rows: {duplicate_count}")


Before we begin, let's quickly define a function to generate summary statistics for a given - this will make analysis easier and more efficient.

In [ ]:
# define calculate stats function
def calculate_stats(data, columns):
    if isinstance(columns, str):
        columns = [columns]

    stats = []
    for col in columns:
        if data[col].dtype in ['object', 'category']:
            counts = data[col].value_counts(dropna=False, sort=False)
            percents = data[col].value_counts(normalize=True, dropna=False, sort=False) * 100
            formatted = counts.astype(str) + ' (' + percents.round(2).astype(str) + '%)'
            stats_col = pd.DataFrame({'count (%)': formatted})
            stats.append(stats_col)
        else:
            stats_col = data[col].describe().to_frame().transpose()
            stats_col['missing'] = data[col].isnull().sum()
            stats_col.index.name = col
            stats.append(stats_col)

    return pd.concat(stats, axis=0)

### Analyzing the Target Variable

Let's conduct some simple analysis on the SII and PCIAT variables.

In [ ]:
# define variables for plotting
sii_counts = train['sii'].value_counts().reset_index()
sii_counts.columns = ['SII', 'Count']
total = sii_counts['Count'].sum()
sii_counts['percentage'] = (sii_counts['Count'] / total) * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# custom color pallete (used throughout)
custom_palette = ['#FFFB46', '#E2A0FF', '#B7FFD8', '#FFC1CF', '#C4F5FC']

axes[0].pie(
    sii_counts['Count'],
    labels=sii_counts['SII'],
    autopct='%1.1f%%',
    colors=custom_palette,
    startangle=140,
    wedgeprops={'edgecolor': 'black'}
)
axes[0].set_title('Distribution of Severity Impairment Index (SII)', fontsize=14)
axes[0].axis('equal')  # Equal aspect ratio makes the pie a circle

# PCIAT_Total for complete responses
sns.histplot(train['complete_resp_total'].dropna(), bins=20, ax=axes[1])
axes[1].set_title('Distribution of PCIAT_Total', fontsize=14)
axes[1].set_xlabel('PCIAT_Total for Complete PCIAT Responses')

plt.tight_layout()
plt.show()

We can notice that 60% of participants are supposedly not affected by internet use, 26% are mildly impaired, and only a little over 1% are severely impaired. PCIAT scores seem to mostly follow a normal curve, though with many participants answering 0 on all questions - with reasonable assurance, we can assume some of these responses are disingenuous. 

Let us now use our handy *calculate_stats* funtion to take a look at some essential demographics for participants in the study, age, sex, and SII distribution across these demographics.

In [ ]:
assert train['Basic_Demos-Age'].isna().sum() == 0
assert train['Basic_Demos-Sex'].isna().sum() == 0

# participant age distribution
train['Age Group'] = pd.cut(train['Basic_Demos-Age'], bins=[4, 12, 18, 22], labels=['Children (5-12)', 'Adolescents (13-18)', 'Adults (19-22)'])
calculate_stats(train, 'Age Group')

In [ ]:
# participant sex distribution
sex_map = {0: 'Male', 1: 'Female'}
train['Basic_Demos-Sex'] = train['Basic_Demos-Sex'].map(sex_map)
calculate_stats(train, 'Basic_Demos-Sex')

In [ ]:
# SII distribution by age group
stats_age = train.groupby(['Age Group', 'sii'], observed=False).size().unstack(fill_value=0)
stats_age_prop = stats_age.div(stats_age.sum(axis=1), axis=0) * 100
stats_age = stats_age.astype(str) +' (' + stats_age_prop.round(1).astype(str) + '%)'
stats_age

In [ ]:
# SII distribution by sex
stats_sex = train.groupby(['Basic_Demos-Sex', 'sii'], observed=False).size().unstack(fill_value=0)
stats_sex_prop = stats_sex.div(stats_sex.sum(axis=1), axis=0) * 100
stats_sex = stats_sex.astype(str) +' (' + stats_sex_prop.round(1).astype(str) + '%)'
stats_sex

The tables show that the distribution of SII scores for children and adults is skewed towards lower values, while scores for adolescents are relatively balanced across the categories. We also might notice observe that there are significantly more severe cases for males, although this is slightly misleading since the test sample is heavily skewed male - in actuality, the differences between SII distribution in males and females are not significant.

### Internet Use and SII

Now let us take a closer look at internet usage specifically to see how it relates to SII.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# generate bar graph, distribution of hours of internet use
ax1 = sns.countplot(x='PreInt_EduHx-computerinternet_hoursday', hue='PreInt_EduHx-computerinternet_hoursday', data=train, palette=custom_palette[:4], legend=False, ax=axes[0], edgecolor='black', linewidth=0.6)
axes[0].set_title('Distribution of Hours of Internet Use')
axes[0].set_xlabel('Hours per Day Group')
axes[0].set_ylabel('Count')

total = len(train['PreInt_EduHx-computerinternet_hoursday'])
for p in ax1.patches:
    count = int(p.get_height())
    percentage = '{:.1f}%'.format(100 * count / total)
    ax1.annotate(f'{count} ({percentage})', (p.get_x() + p.get_width() / 2., p.get_height()), 
                 ha='center', va='baseline', fontsize=10, color='black', xytext=(0, 5), 
                 textcoords='offset points')

# generate boxplot, hours of internet use by age
sns.boxplot(y=train['Basic_Demos-Age'], x=train['PreInt_EduHx-computerinternet_hoursday'], hue=train['PreInt_EduHx-computerinternet_hoursday'], palette=custom_palette[:4], ax=axes[1], legend=False)
axes[1].set_title('Hours of Internet Use by Age')
axes[1].set_ylabel('Age')
axes[1].set_xlabel('Hours per Day Group')

# generate boxplot, internet hours by age group
sns.boxplot(y='PreInt_EduHx-computerinternet_hoursday', x='Age Group', hue='Age Group',data=train, palette=custom_palette[:3], ax=axes[2], legend=False)
axes[2].set_title('Internet Hours by Age Group')
axes[2].set_ylabel('Hours per Day (Numeric)')
axes[2].set_xlabel('Age Group');

Similar to the earlier SII data, we can see that higher daily internet usage is associated with older age. This hints towards a correlation between internet use and SII.

In [ ]:
fig = plt.figure(figsize=(12, 10))
gs = fig.add_gridspec(2, 2, height_ratios=[1, 1.5])

# generate boxplot, SII by hours of internet use
ax1 = fig.add_subplot(gs[0, 0])
sns.boxplot(x='sii', y='PreInt_EduHx-computerinternet_hoursday', hue='sii', data=train, ax=ax1, palette=custom_palette[:4], legend=False)
ax1.set_title('SII by Hours of Internet Use')
ax1.set_ylabel('Hours per Day')
ax1.set_xlabel('SII')

# generate boxplot, PCIAT_Total by hours of internet use
ax2 = fig.add_subplot(gs[0, 1])
sns.boxplot(x='PreInt_EduHx-computerinternet_hoursday', y='complete_resp_total', hue='PreInt_EduHx-computerinternet_hoursday', data=train, palette=custom_palette[:4], ax=ax2, legend=False)
ax2.set_title('PCIAT_Total by Hours of Internet Use')
ax2.set_ylabel('PCIAT_Total for Complete PCIAT Responses')
ax2.set_xlabel('Hours per Day Group');

Simply tracing your finger along across these plots, a slight upward trend is visible between SII/ PCIAT and hours of internet use, showing their correlation. It might be notable that there are plenty of participants across all age groups that report less than an hour of internet use and still having a high SII score, suggesting that only an hour of internet use can be enough to negatively impact mental health. 

## Exploring Features Related to Physical Activity

A significant portion of the feature data gives information on participants' physical measures, like height, weight, BMI, heart rate, etc. The relationship between internet use and SII is intuitive, but physical characteristics? Results in this analysis might yield some interesting and unexpected results.

There just so happen to be 7 primary physical features in the train set - this is the perfect number of features for building a **correlation matrix**, where each feature is given a score from -1 to 1 based on its correlation with the target variable.

In [ ]:
# covert SII values to float
train['sii'] = train['sii'].str[0].astype(float)

# identify physical measures
cols = ['Physical-BMI', 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference', 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP']
data_subset = train[cols + ['sii']]

# generate correlation matrix
corr_matrix = data_subset.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.show()

This matrix gives a pretty comprehensive overview of how physical features are interrelated - in the top right corener, we see BMI, height, weight and waist circumference all highly correlated with one another, as we migtht expect. But what we're really interested in is how these features are related with SII.

Reading the bottom row, we see that height, weight and waist circumference are positively correlated with SII, while dialostic blood pressure, systolic blood pressure and heart rate are weakly correlated. This would suggest that cardiovascular health is not a determining factor for SII score, although it should be noted that the positive correlation could actually our previously observed age-correlation in disguise, since physical metrics increase with age.

Let us take a look at height and weight by age. We can assume BMI will follow from these.

In [ ]:
plt.figure(figsize=(18, 5))

# physical height by age
plt.subplot(1, 3, 1)
sns.scatterplot(x='Basic_Demos-Age', y='Physical-Height', data=train)
plt.title('Physical-Height by Age')
plt.xlabel('Age')
plt.ylabel('Height (cm)')

# physical weight by age
plt.subplot(1, 3, 2)
sns.scatterplot(x='Basic_Demos-Age', y='Physical-Weight', data=train)
plt.title('Physical-Weight by Age')
plt.xlabel('Age')
plt.ylabel('Weight (kg)')

plt.tight_layout()
plt.show()

Unsurprisingly, physical metrics are positively correlated with age. We can also see a number of outliers in the weight measurements, which might suggest a slight increase in correlation between weight and SII. Comparing physical metrics with SII directly, we get the following.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# boxplot: SII vs BMI
sns.boxplot(x='sii', y='Physical-BMI', data=train, hue='sii', ax=axes[0], palette=custom_palette[:4], legend=False)
axes[0].set_title('SII vs BMI')
axes[0].set_xlabel('SII Category')
axes[0].set_ylabel('BMI')

# boxplot: SII vs Height
sns.boxplot(x='sii', y='Physical-Height', data=train, hue='sii', ax=axes[1], palette=custom_palette[:4], legend=False)
axes[1].set_title('SII vs Height')
axes[1].set_xlabel('SII Category')
axes[1].set_ylabel('Height (cm)')

# boxplot: SII vs Weight
sns.boxplot(x='sii', y='Physical-Weight', data=train, hue='sii', ax=axes[2], palette=custom_palette[:4], legend=False)
axes[2].set_title('SII vs Weight')
axes[2].set_xlabel('SII Category')
axes[2].set_ylabel('Weight (kg)')

plt.tight_layout()
plt.show()

Here we can see something kind of interesting - there's a pretty sharp jump in BMI and weight statistics when SII is 3. This jump is less pronounced with height, seeming to suggest that BMI increases through its relation to weight specifically. This gives us a clue that **weight** might play a significant factor in higher SII scores, which makes intuitive sense.

Let us now take a look at the top 20 features based on correlation with the target variable.

In [ ]:
# isolate non-PCIAT features
train_subset = train.drop(train.columns[55:76].tolist() + ['complete_resp_total'], axis=1)
numerical_features = train_subset.select_dtypes(include=[np.number]).columns.tolist()

# calculate Pearson correlations with 'sii'
target_correlations = []
for feature in numerical_features:
    if feature != 'sii':
        corr = train_subset[[feature, 'sii']].corr().iloc[0, 1]
        if not np.isnan(corr):
            target_correlations.append((feature, abs(corr)))

# sort and take top 20
top_corr = sorted(target_correlations, key=lambda x: x[1], reverse=True)[:20]
features, corrs = zip(*top_corr)

# plot features
plt.figure(figsize=(12, 8))
plt.barh(range(len(features)), corrs, color='#E2A0FF', edgecolor='black')
plt.yticks(range(len(features)), features)
plt.xlabel('Absolute Correlation with SII')
plt.title('Top 20 Features by Correlation with SII')
plt.gca().invert_yaxis()  # Optional: highest at top
plt.tight_layout()
plt.show()

As we found earlier, features like height, age, internet use and weight top the list. We also see positive correlations with other features we did not look as closely into - these other features will play a role during the feature engineering and selection process.

Last thing we'll do for this section is remove all features that are directly related to SII, like PCIAT scores, and arbitrary identifiers like "id". 


# Model Training 🏋

With an strong understanding of the data at hand, we are ready to tackle model training! 🎉

Since we're attacking this problem with an ensemble appraoch, this means creating separate pipelines for pre-processing, feature engineering, training and validation. Will this add an extra layer of complexity? Yes. But, it will get us closer to an accurate solution, which is not biased to any one model, increasing our chances of an accurate prediction. We're going to train a **Ridge model**, which is a linear regression model intended for interpretability, and a **LightGBM model**, a slightly more advanced model that can handle non-linear relationships and complex interactions between features - together, these models should provide a balanced foundation that we can tune and optimize. Here is a basic workflow for this section:

*‣ General pre-processing for all data*

*‣ General feature engineering for all data based on EDA and correlation*

*‣ Split data into "train" and "validation" set*

*‣ Ridge model training: feature selection, train and validate on validation set with various correlation thresholds, then select best model based on RMSE (Root Mean Squared Error)*

*‣ LightGBM model training: rank features based on importance, define model parameters,train and validate on validation set*

*‣ Conduct analysis on the results of our training and prepare for model tuning*

First, let's remove all features that are directly related to SII, like PCIAT scores, and arbitrary identifiers like "id". This will prevent data leakage and possible overfitting.

In [ ]:
# remove all PCIAT columns
pciat_cols = [col for col in train.columns if 'PCIAT' in col]
train = train.drop(columns=pciat_cols)

# remove "complete_resp" and "age group" columns
train = train.drop("complete_resp_total", axis=1)
train = train.drop("Age Group", axis=1) 

# remove id column
train = train.drop("id", axis=1)

# train.to_csv("/Users/tomragus/Library/CloudStorage/OneDrive-UCSanDiego/CMI-PIU-Model/data/train_cleaned.csv", index=False)

Next, let's define some variables, separate the target variable from the dataset, and re-calculate Pearson correlations - these correlations will come in handy for feature selection.

In [ ]:
# separate target variable from features
X = train.drop('sii', axis=1)
y = train['sii']

# sets for categorical and numerical features
categorical_cols = []
numerical_cols = []

for col in X.columns:
    if X[col].dtype == 'object':
        categorical_cols.append(col)
    else:
        numerical_cols.append(col)

# re-calculate Pearson correlations with target for numerical features
def calculate_correlations(X, y, numerical_cols):
    correlations = {}
    for col in numerical_cols:
        mask = ~(X[col].isnull() | y.isnull())
        if mask.sum() > 1:
            corr, p_value = pearsonr(X[col][mask], y[mask])
            correlations[col] = {'correlation': corr, 'p_value': p_value}
    return correlations

correlations = calculate_correlations(X, y, numerical_cols)

# sort by absolute correlation
sorted_correlations = sorted(correlations.items(), 
                           key=lambda x: abs(x[1]['correlation']), 
                           reverse=True)

print("\nTop 20 correlations with SII:")
for i, (col, stats) in enumerate(sorted_correlations[:20]):
    print(f"{i+1:2d}. {col:<40} | Corr: {stats['correlation']:6.3f} | p-value: {stats['p_value']:.4f}")

Here we will define some new features, some of my own creation, and some based on our calculated correlations. These new features, on top of the existing features, will give the models a lot to work with.

‣ Difference between BMI and BIA

‣ Total exercise duration

‣ Ratio of Fat to FFM

‣ Z-score normalation of cardiovascular health statistics (heart rate, systolic and diastolic blood pressure)

‣ Age squared

‣ Age group

In [ ]:
# defining feature engineering functions
def create_engineered_features(df, correlations=None):
    df_eng = df.copy()
    
    # BMI-related features
    if 'Physical-BMI' in df_eng.columns and 'BIA-BIA_BMI' in df_eng.columns:
        df_eng['BMI_difference'] = df_eng['Physical-BMI'] - df_eng['BIA-BIA_BMI']
    
    # fitness ratios
    if 'Fitness_Endurance-Time_Mins' in df_eng.columns and 'Fitness_Endurance-Time_Sec' in df_eng.columns:
        df_eng['Total_Fitness_Time'] = df_eng['Fitness_Endurance-Time_Mins'] * 60 + df_eng['Fitness_Endurance-Time_Sec']
    
    # body composition ratios
    if 'BIA-BIA_Fat' in df_eng.columns and 'BIA-BIA_FFM' in df_eng.columns:
        df_eng['Fat_to_FFM_ratio'] = df_eng['BIA-BIA_Fat'] / (df_eng['BIA-BIA_FFM'] + 1e-8)
    
    # physical health composite
    if all(col in df_eng.columns for col in ['Physical-HeartRate', 'Physical-Systolic_BP', 'Physical-Diastolic_BP']):
        health_cols = ['Physical-HeartRate', 'Physical-Systolic_BP', 'Physical-Diastolic_BP']
        for col in health_cols:
            if df_eng[col].notna().sum() > 0:
                mean_val = df_eng[col].mean()
                std_val = df_eng[col].std()
                df_eng[f'{col}_normalized'] = (df_eng[col] - mean_val) / (std_val + 1e-8)
    
    # age-related
    if 'Basic_Demos-Age' in df_eng.columns:
        df_eng['Age_squared'] = df_eng['Basic_Demos-Age'] ** 2
        df_eng['Age_group'] = pd.cut(df_eng['Basic_Demos-Age'], 
                                   bins=[0, 8, 12, 16, 22], 
                                   labels=['child', 'preteen', 'teen', 'young_adult'])
    
    # create correlation-based features
    if correlations is not None:
        top_corr_features = [col for col, stats in sorted(correlations.items(), key=lambda x: abs(x[1]['correlation']), reverse=True)[:10] if col in df_eng.columns]
        
        for i, col1 in enumerate(top_corr_features[:5]):
            for col2 in top_corr_features[i+1:5]:
                if col1 in df_eng.columns and col2 in df_eng.columns:
                    if df_eng[col1].dtype in ['int64', 'float64'] and df_eng[col2].dtype in ['int64', 'float64']:
                        df_eng[f'{col1}_x_{col2}'] = df_eng[col1] * df_eng[col2]
    
    return df_eng

# apply feature engineering
X_engineered = create_engineered_features(X, correlations)

print("New features created:")
new_features = set(X_engineered.columns) - set(X.columns)
for feat in new_features:
    print(f"  - {feat}")

Keeping track of and separating data by datatype will allow our models to digest the data, so here we will organize data by type - we will also recalculate Pearson correlation, now that we have additional features to account for.

In [ ]:
# re-create categorical and numerical sets with engineered features
categorical_cols_eng = []
numerical_cols_eng = []

for col in X_engineered.columns:
    if is_object_dtype(X_engineered[col]) or isinstance(X_engineered[col].dtype, CategoricalDtype):
        categorical_cols_eng.append(col)
    elif is_numeric_dtype(X_engineered[col]):
        numerical_cols_eng.append(col)

print(f"\nFinal categorical columns: {len(categorical_cols_eng)}")
print(f"Final numerical columns: {len(numerical_cols_eng)}")

# recalculate correlations with engineered features
correlations_eng = calculate_correlations(X_engineered, y, numerical_cols_eng)

# sort by absolute correlation
sorted_correlations_eng = sorted(correlations_eng.items(), key=lambda x: abs(x[1]['correlation']), reverse=True)

print("\nTop 15 correlations with SII (after feature engineering):")
for i, (col, stats) in enumerate(sorted_correlations_eng[:15]):
    print(f"{i+1:2d}. {col:<40} | Corr: {stats['correlation']:6.3f} | p-value: {stats['p_value']:.4f}")

Notice, every feature with "_x_" is one of the features created based on correlations with eachother. And look at that! A significant portion of our features with the highest correlation with SII are these new created features. You can also features like "age squared" making their way into the list. Now, we have a wide selection of **highly correlated** features to work with.

We're almost ready to train the models - but before we do, let us split our data into training and validation sets.

In [ ]:
# split engineered data into train and validation sets (80/20 split)
X_train, X_val, y_train, y_val = train_test_split(
    X_engineered, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")

Time for training! 🏋️

For the Ridge model, we'll want to do a few things. **First,** we'll want to select the features to use for the model. We'll do this by using the correlation filtering method. **Second,** we will account for missing values in the dataset, filling empty numerical data with the median, and empty categorical data with the mode. **Third,** we will scale the features to have zero mean and unit variance, and encode the categorical variables using one-hot encoding. We'll train the models with a few different **correlation thresholds** (meaning, only features meeting a certain correlation threshold will be included in the training), as well as one with no threshold, and for each evaluate the model on our validation set, selecting the model with the **lowest validation RMSE.** 

The thresholds I will use are 0.01, 0.03, 0.05, and 0.1.

In [ ]:
# RIDGE REGRESSION PREPROCESSING PIPELINE
print("\n" + "="*50)
print("RIDGE REGRESSION PREPROCESSING")
print("="*50)

# define function for feature selection based on threshold
def select_features_by_correlation(X, y, numerical_cols, threshold=0.05):
    correlations = calculate_correlations(X, y, numerical_cols)
    
    selected_features = []
    for col, stats in correlations.items():
        if abs(stats['correlation']) >= threshold and stats['p_value'] < 0.05:
            selected_features.append(col)
    
    return selected_features

# define ridge model preprocessing pipeline
def preprocess_for_ridge(X_train, X_val, categorical_cols, numerical_cols, correlation_threshold=0.05, use_correlation_filtering=True):
    
    # create copies of data
    X_train_processed = X_train.copy()
    X_val_processed = X_val.copy()
    
    # handle missing values (fill numerical with median, categorical with mode)
    for col in numerical_cols:
        if col in X_train_processed.columns:
            median_val = X_train_processed[col].median()
            X_train_processed[col] = X_train_processed[col].fillna(median_val)
            X_val_processed[col] = X_val_processed[col].fillna(median_val)
    
    for col in categorical_cols:
        if col in X_train_processed.columns:
            mode_val = X_train_processed[col].mode()[0] if not X_train_processed[col].mode().empty else 'Unknown'
            X_train_processed[col] = X_train_processed[col].fillna(mode_val)
            X_val_processed[col] = X_val_processed[col].fillna(mode_val)
    
    # feature selection based on correlation
    selected_numerical_features = numerical_cols
    if use_correlation_filtering:
        selected_numerical_features = select_features_by_correlation(
            X_train_processed, y_train, numerical_cols, correlation_threshold
        )
        
        print(f"Selected {len(selected_numerical_features)} numerical features out of {len(numerical_cols)} based on correlation > {correlation_threshold}")
        print("Selected features:", selected_numerical_features[:10], "..." if len(selected_numerical_features) > 10 else "")
        
        # keep only selected numerical features
        features_to_keep = selected_numerical_features + categorical_cols
        X_train_processed = X_train_processed[features_to_keep]
        X_val_processed = X_val_processed[features_to_keep]
    
    # one-hot encoding for categorical variables
    X_train_encoded = pd.get_dummies(X_train_processed, columns=categorical_cols, drop_first=True)
    X_val_encoded = pd.get_dummies(X_val_processed, columns=categorical_cols, drop_first=True)
    
    # validate continuity between sets
    missing_cols_val = set(X_train_encoded.columns) - set(X_val_encoded.columns)
    missing_cols_train = set(X_val_encoded.columns) - set(X_train_encoded.columns)
    
    for col in missing_cols_val:
        X_val_encoded[col] = 0
    for col in missing_cols_train:
        X_train_encoded[col] = 0
    
    X_val_encoded = X_val_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
    
    # scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_encoded)
    X_val_scaled = scaler.transform(X_val_encoded)
    
    encoded_cols = list(X_train_encoded.columns)

    return X_train_scaled, X_val_scaled, scaler, encoded_cols, selected_numerical_features

# define different correlation thresholds
correlation_thresholds = [0.01, 0.03, 0.05, 0.1]
ridge_results = {}

print("\nTesting different correlation thresholds for Ridge:")
for threshold in correlation_thresholds:
    print(f"\n--- Testing threshold: {threshold} ---")
    
    # preprocessing
    X_train_ridge, X_val_ridge, scaler, encoded_cols, selected_features = preprocess_for_ridge(
        X_train, X_val, categorical_cols_eng, numerical_cols_eng, 
        correlation_threshold=threshold, use_correlation_filtering=True
    )
    
    print(f"Ridge - Train shape after preprocessing: {X_train_ridge.shape}")
    
    # train Ridge Regression
    ridge_model = Ridge(alpha=1.0, random_state=42)
    ridge_model.fit(X_train_ridge, y_train)
    
    # predictions and evaluation
    y_train_pred_ridge = ridge_model.predict(X_train_ridge)
    y_val_pred_ridge = ridge_model.predict(X_val_ridge)
    
    ridge_train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred_ridge))
    ridge_val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_ridge))
    
    ridge_results[threshold] = {
        'train_rmse': ridge_train_rmse,
        'val_rmse': ridge_val_rmse,
        'num_features': X_train_ridge.shape[1],
        'selected_features': selected_features
    }
    
    print(f"Train RMSE: {ridge_train_rmse:.4f}")
    print(f"Validation RMSE: {ridge_val_rmse:.4f}")

# testing without correlation fitting
print(f"\n--- Testing without correlation filtering ---")
X_train_ridge_no_filter, X_val_ridge_no_filter, ridge_scaler_no_filter, ridge_columns_no_filter, _ = preprocess_for_ridge(
    X_train, X_val, categorical_cols_eng, numerical_cols_eng, 
    use_correlation_filtering=False
)

print(f"Ridge - Train shape after preprocessing: {X_train_ridge_no_filter.shape}")

ridge_model_no_filter = Ridge(alpha=1.0, random_state=42)
ridge_model_no_filter.fit(X_train_ridge_no_filter, y_train)

y_train_pred_ridge_no_filter = ridge_model_no_filter.predict(X_train_ridge_no_filter)
y_val_pred_ridge_no_filter = ridge_model_no_filter.predict(X_val_ridge_no_filter)

ridge_train_rmse_no_filter = np.sqrt(mean_squared_error(y_train, y_train_pred_ridge_no_filter))
ridge_val_rmse_no_filter = np.sqrt(mean_squared_error(y_val, y_val_pred_ridge_no_filter))

ridge_results['no_filter'] = {
    'train_rmse': ridge_train_rmse_no_filter,
    'val_rmse': ridge_val_rmse_no_filter,
    'num_features': X_train_ridge_no_filter.shape[1],
    'selected_features': None
}

print(f"Train RMSE: {ridge_train_rmse_no_filter:.4f}")
print(f"Validation RMSE: {ridge_val_rmse_no_filter:.4f}")

# selecting best Ridge configuration for final model
best_ridge_config = min(ridge_results.items(), key=lambda x: x[1]['val_rmse'])
print(f"\nBest Ridge configuration: {best_ridge_config[0]} (Validation RMSE: {best_ridge_config[1]['val_rmse']:.4f})")

if best_ridge_config[0] == 'no_filter':
    X_train_ridge_final, X_val_ridge_final = X_train_ridge_no_filter, X_val_ridge_no_filter
    ridge_model_final = ridge_model_no_filter
    final_ridge_train_rmse, final_ridge_val_rmse = ridge_train_rmse_no_filter, ridge_val_rmse_no_filter
else:
    X_train_ridge_final, X_val_ridge_final, _, _, _ = preprocess_for_ridge(
        X_train, X_val, categorical_cols_eng, numerical_cols_eng, 
        correlation_threshold=best_ridge_config[0], use_correlation_filtering=True
    )
    ridge_model_final = Ridge(alpha=1.0, random_state=42)
    ridge_model_final.fit(X_train_ridge_final, y_train)
    
    y_train_pred_final = ridge_model_final.predict(X_train_ridge_final)
    y_val_pred_final = ridge_model_final.predict(X_val_ridge_final)
    
    final_ridge_train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred_final))
    final_ridge_val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_final))

BOOM! 💣

We have our 5 models, one for each threshold and one with no threshold. Observing the results, we can see training with a testing threshold of 0.1 gives us the lowest validation RMSE of 0.6768. *How do we interpret this result?* This means that on average, when predicting SII (recall SII ranges from 0 to 3), our model is off by 0.6768 points. Not bad - there is room for improvement, but the model is frequently correct or close to correct. In this result we also have a train RMSE of 0.663. This slight difference between RMSE scores suggest slight overfitting, meaning the model performs better on the training data than it does on the validation data. 

Now time to tackle the LightGBM model - just like with Ridge, we'll fill missing values and split into train and validation sets. One thing about LightGBM is that it automatically assigns an "immportance" value to each feature, removing the need to utilize correlation filtering. Many of the things we did manually for Ridge, LightGBM does internally - all we do is tell LigthGBM what parameters to use, and it does the rest (rewrite this part EW!!!)

In [ ]:
# LIGHTGBM PREPROCESSING PIPELINE
print("\n" + "="*50)
print("LIGHTGBM PREPROCESSING")
print("="*50)

def preprocess_for_lightgbm(X_train, X_val, categorical_cols, numerical_cols):
    
    # create copies of data
    X_train_processed = X_train.copy()
    X_val_processed = X_val.copy()
    
    # handle missing values
    for col in numerical_cols:
        if col in X_train_processed.columns:
            median_val = X_train_processed[col].median()
            X_train_processed[col] = X_train_processed[col].fillna(median_val)
            X_val_processed[col] = X_val_processed[col].fillna(median_val)
    
    # label encoding for categorical variables
    label_encoders = {}
    for col in categorical_cols:
        if col in X_train_processed.columns:
            le = LabelEncoder()
            
            # fill missing values first
            mode_val = X_train_processed[col].mode()[0] if not X_train_processed[col].mode().empty else 'Unknown'
            X_train_processed[col] = X_train_processed[col].fillna(mode_val)
            X_val_processed[col] = X_val_processed[col].fillna(mode_val)

            # fit on train data
            X_train_processed[col] = le.fit_transform(X_train_processed[col]).astype(int)

            # handle unseen categories in validation
            train_categories = set(le.classes_)
            X_val_processed[col] = X_val_processed[col].map(
                lambda x: le.transform([x])[0] if x in train_categories else 0
            ).astype(int)

            label_encoders[col] = le
    
    return X_train_processed, X_val_processed, label_encoders

# preprocess for LightGBM
X_train_lgb, X_val_lgb, lgb_encoders = preprocess_for_lightgbm(
    X_train, X_val, categorical_cols_eng, numerical_cols_eng
)

categorical_feature_names = [col for col in categorical_cols_eng if col in X_train_lgb.columns]

print(f"LightGBM - Train shape after preprocessing: {X_train_lgb.shape}")
print(f"LightGBM - Validation shape after preprocessing: {X_val_lgb.shape}")

# define LightGBM parameters
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'colsample_bytree': 0.9,
    'subsample': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'random_state': 42,
    'force_col_wise': True
}

# create LightGBM datasets
train_data = lgb.Dataset(X_train_lgb, label=y_train, categorical_feature=categorical_feature_names)
val_data = lgb.Dataset(X_val_lgb, label=y_val, reference=train_data, categorical_feature=categorical_feature_names)

# train LightGBM model
lgb_model = lgb.train(
    lgb_params,
    train_data,
    valid_sets=[train_data, val_data],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(0)]
)

# predictions and evaluation
y_train_pred_lgb = lgb_model.predict(X_train_lgb, num_iteration=lgb_model.best_iteration)
y_val_pred_lgb = lgb_model.predict(X_val_lgb, num_iteration=lgb_model.best_iteration)

lgb_train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred_lgb))
lgb_val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_lgb))

print(f"\nLightGBM Results:")
print(f"Train RMSE: {lgb_train_rmse:.4f}")
print(f"Validation RMSE: {lgb_val_rmse:.4f}")

With LightGBM, we get a slightly lower validation RMSE of 0.6693. Again, a decent score! This time however, we notice a more significant difference between the train RMSE of 0.5296 and the validation RMSE, which is evidence of overfitting. 

Now with both of our models trained, let's compare their performance.

In [ ]:
# generate final model comparison
print(f"\nFinal Ridge Regression (Best Config: {best_ridge_config[0]}):")
print(f"  Train RMSE: {final_ridge_train_rmse:.4f}")
print(f"  Validation RMSE: {final_ridge_val_rmse:.4f}")
print(f"  Overfitting: {final_ridge_val_rmse - final_ridge_train_rmse:.4f}")

print(f"\nLightGBM (All Features):")
print(f"  Train RMSE: {lgb_train_rmse:.4f}")
print(f"  Validation RMSE: {lgb_val_rmse:.4f}")
print(f"  Overfitting: {lgb_val_rmse - lgb_train_rmse:.4f}")

# generate features selected for Ridge
if best_ridge_config[0] != 'no_filter':
    print(f"\nFeatures selected for Ridge (correlation > {best_ridge_config[0]}):")
    selected_features = best_ridge_config[1]['selected_features']
    for i, feat in enumerate(selected_features[:15]):  # Show first 15
        corr_val = correlations_eng.get(feat, {}).get('correlation', 0)
        print(f"{i+1:2d}. {feat:<40} | Corr: {corr_val:6.3f}")
    if len(selected_features) > 15:
        print(f"... and {len(selected_features) - 15} more features")

# generate feature importance for LightGBM
print(f"\nTop 10 Most Important Features (LightGBM):")
feature_importance = lgb_model.feature_importance(importance_type='gain')
feature_names = X_train_lgb.columns
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

for i, (_, row) in enumerate(importance_df.head(10).iterrows()):
    print(f"{i+1:2d}. {row['feature']:<40} | Importance: {row['importance']:8.0f}")

# generate overfitting analysis
print("\nOverfitting Analysis:")
ridge_overfit = final_ridge_val_rmse - final_ridge_train_rmse
lgb_overfit = lgb_val_rmse - lgb_train_rmse
print(f"   - Ridge overfitting: {ridge_overfit:.4f}")
print(f"   - LightGBM overfitting: {lgb_overfit:.4f}")
if ridge_overfit < lgb_overfit:
    print("   → Ridge is more robust (less overfitting)")
else:
    print("   → LightGBM is more robust (less overfitting)")

Here are some key takeaways:

*‣ Both models preformed fairly well on the validation set, with the Ridge model performing slightly better.*

*‣ Looking at the features ranked by correlation for the Ridge model and comparing with the features ranked by importance by the LightGBM model, we see many of the same features high on the list, like Age, Physical Weight, etc. This is good and shows a level of consistency between the models.*

*‣ Both models show signs of overfitting, with the effect being more pronounced in the LightGBM model. This will be a key point of focus during the tuning phase.*

blah blah blah (leading into tuning)

## Tuning Our Models 📻

blah blah blah

In [ ]:
# Perform Ridge alpha tuning after selecting best preprocessing config
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge = Ridge(random_state=42)
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_root_mean_squared_error')
grid_search.fit(X_train_ridge_final, y_train)

print(f"Best alpha: {grid_search.best_params_['alpha']}")

# Retrain with best alpha from CV
best_alpha = grid_search.best_params_['alpha']

ridge_model_final = Ridge(alpha=best_alpha, random_state=42)
ridge_model_final.fit(X_train_ridge_final, y_train)

# Evaluate on validation set
y_train_pred = ridge_model_final.predict(X_train_ridge_final)
y_val_pred = ridge_model_final.predict(X_val_ridge_final)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

print(f"\nFinal Ridge Model with alpha={best_alpha}:")
print(f"  Train RMSE: {train_rmse:.4f}")
print(f"  Validation RMSE: {val_rmse:.4f}")

blah blah blah

In [ ]:
param_grid = {
    'learning_rate': [0.01, 0.05],
    'num_leaves': [15, 31],
    'max_depth': [-1, 5],
    'colsample_bytree': [0.8, 1.0],
    'subsample': [0.8, 1.0],
    'min_child_samples': [20, 50]
}

lgb_params['verbose'] = -1

# Define base model
lgb_model = LGBMRegressor(
    objective='regression',
    random_state=42,
    n_estimators=1000  # early stopping will prevent overtraining
)

grid_search = GridSearchCV(
    estimator=lgb_model,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',  # RMSE
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1
)

# Fit on your preprocessed LightGBM training data
grid_search.fit(X_train_lgb, y_train)

print(f"Best Parameters:\n{grid_search.best_params_}")
print(f"Best CV RMSE: {-grid_search.best_score_:.4f}")

best_lgb_params = grid_search.best_params_

# Final model with tuned params
final_lgb_model = LGBMRegressor(
    **best_lgb_params,
    objective='regression',
    random_state=42,
    n_estimators=1000
)

# Fit on full training set with early stopping on validation
final_lgb_model.fit(
    X_train_lgb, y_train,
    eval_set=[(X_val_lgb, y_val)],
    eval_metric='rmse',
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(0)  # 0 disables log output
    ]
)

# Predict and evaluate
y_train_pred = final_lgb_model.predict(X_train_lgb)
y_val_pred = final_lgb_model.predict(X_val_lgb)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

print(f"\nTuned LightGBM Results:")
print(f"  Train RMSE: {train_rmse:.4f}")
print(f"  Validation RMSE: {val_rmse:.4f}")

## FINAL MODEL RESULTS:

‣ Ridge Regression: 0.6710

‣ LightGBM: 0.6657

## Training the Test Set: Ridge Model

blah blah blah

In [ ]:
# convert sex binary to string
test['Basic_Demos-Sex'] = test['Basic_Demos-Sex'].map({0: 'Male', 1: 'Female'})

# separate id from test set
test_ids = test["id"].copy()
test = test.drop("id", axis=1)

# update test set with engineered features for Ridge model
X_test_engineered = create_engineered_features(test, correlations)

def clip_predictions(predictions):
    # Round predictions to nearest integer
    rounded = np.round(predictions)
    # Clip values to be between 0 and 3
    clipped = np.clip(rounded, 0, 3)
    return clipped

blah blah blah

In [ ]:

def preprocess_test_for_ridge(X_test, categorical_cols, numerical_cols, 
                             selected_numerical_features, scaler, encoded_column_names):
    X_test_processed = X_test.copy()
    
    # handle missing values (fill numerical with median, categorical with mode)
    for col in numerical_cols:
        if col in X_test_processed.columns:
            median_val = X_test_processed[col].median()
            X_test_processed[col] = X_test_processed[col].fillna(median_val)
    
    for col in categorical_cols:
        if col in X_test_processed.columns:
            mode_val = X_test_processed[col].mode()[0] if not X_test_processed[col].mode().empty else 'Unknown'
            X_test_processed[col] = X_test_processed[col].fillna(mode_val)

    # Keep only the features selected during training
    features_to_keep = selected_numerical_features + categorical_cols
    X_test_processed = X_test_processed[features_to_keep]

    # One-hot encode using same method
    X_test_encoded = pd.get_dummies(X_test_processed, columns=categorical_cols, drop_first=True)

    # Add any missing columns (present in training but not in test)
    for col in encoded_column_names:
        if col not in X_test_encoded.columns:
            X_test_encoded[col] = 0

    # Reorder columns to match training
    X_test_encoded = X_test_encoded[encoded_column_names]

    # Scale using training scaler
    X_test_scaled = scaler.transform(X_test_encoded)
    
    return X_test_scaled


X_test_scaled = preprocess_test_for_ridge(
    X_test_engineered,
    categorical_cols = categorical_cols_eng,
    numerical_cols = numerical_cols_eng,
    selected_numerical_features = selected_features,
    scaler = scaler,
    encoded_column_names = encoded_cols
)

predictions_ridge = ridge_model_final.predict(X_test_scaled)
y_pred_ridge = clip_predictions(predictions_ridge)

blah blah blah

## Training the Test Set: LightGBM Model

blah blah blah

In [ ]:
def preprocess_test_for_lightgbm(X_test, categorical_cols, numerical_cols, label_encoders):
    X_test_processed = X_test.copy()
    
    # handle missing values
    for col in numerical_cols:
        if col in X_test_processed.columns:
            median_val = X_test_processed[col].median()
            X_test_processed[col] = X_test_processed[col].fillna(median_val)
    
    # Label encode using training encoders
    for col in categorical_cols:
        if col in X_test_processed.columns:
            le = label_encoders.get(col)
            if le:
                mode_val = X_test_processed[col].mode()[0] if not X_test_processed[col].mode().empty else 'Unknown'
                X_test_processed[col] = X_test_processed[col].fillna(mode_val)

                train_categories = set(le.classes_)
                X_test_processed[col] = X_test_processed[col].map(
                    lambda x: le.transform([x])[0] if x in train_categories else 0
                ).astype(int)
            else:
                X_test_processed[col] = 0  # failsafe for unexpected cases
    
    return X_test_processed

X_test_lgb = preprocess_test_for_lightgbm(
    X_test_engineered,
    categorical_cols = categorical_cols_eng,
    numerical_cols = numerical_cols_eng,
    label_encoders = lgb_encoders
)

predictions_lgb = final_lgb_model.predict(X_test_lgb)
y_pred_lgb = clip_predictions(predictions_lgb)

In [ ]:
y_pred_ensemble = 0.5 * y_pred_ridge + 0.5 * y_pred_lgb
y_pred_final = np.round(y_pred_ensemble).astype(int)

submission = pd.DataFrame({
    'id': test_ids,
    'SII': y_pred_final
})

display(submission)